In [15]:
from elasticsearch import Elasticsearch

# Elasticsearch 클라이언트 설정
es = Elasticsearch("http://localhost:9200")

def query_pharmacy_elasticsearch(user_lat, user_lon):
    """
    Elasticsearch에서 사용자 위치와 가까운 약국 검색.
    """
    query = {
        "query": {
            "bool": {
                "filter": [
                    {
                        "geo_distance": {
                            "distance": "500km",  # 500km 제한
                            "location": {
                                "lat": user_lat,
                                "lon": user_lon
                            }
                        }
                    }
                ]
            }
        },
        "sort": [
            {
                "_geo_distance": {
                    "location": {"lat": user_lat, "lon": user_lon},
                    "order": "asc",
                    "unit": "km"
                }
            }
        ]
    }

    # Elasticsearch 검색 실행
    es_results = es.search(index="pharmacy_records_v2", body=query)
    return es_results


In [16]:
# 네이버 길찾기 api 사용 -> 소요 시간 측정
import requests

def get_travel_time(user_lat, user_lon, pharmacy_lat, pharmacy_lon):
    import requests
    import configparser

    # ConfigParser 초기화
    config = configparser.ConfigParser()

    # keys.config 파일 읽기
    config.read('../keys.config')

    url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": config['API_KEYS']['naver_api_key_id'],
        "X-NCP-APIGW-API-KEY": config['API_KEYS']['naver_api_key']
    }
    params = {
        "start": f"{user_lon},{user_lat}",
        "goal": f"{pharmacy_lon},{pharmacy_lat}",
        "option": "trafast"  # 가장 빠른 경로
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['code'] == 0:
            # 총 소요시간을 초 단위로 반환
            return data['route']['trafast'][0]['summary']['duration'] / 1000
        else:
            print(f"API 응답 오류: {data['message']}")
            return None
    else:
        print(f"HTTP 요청 오류: {response.status_code}")
        return None

In [22]:
def add_travel_times_to_pharmacies(df, user_lat, user_lon):
    """
    DataFrame에 약국 소요시간 정보 추가.
    """
    travel_times = []
    for _, row in df.iterrows():
        pharmacy_lat = row['wgs84lat']
        pharmacy_lon = row['wgs84lon']
        travel_time_sec = get_travel_time(user_lat, user_lon, pharmacy_lat, pharmacy_lon)
        travel_times.append(travel_time_sec)

    # 소요시간 추가
    df['travel_time_sec'] = travel_times
    df['travel_time_h'] = df['travel_time_sec'] // 3600
    df['travel_time_min'] = (df['travel_time_sec'] % 3600) // 60
    return df

In [23]:
import pandas as pd

if __name__ == "__main__":
    # 사용자 입력
    user_lat = 37.545216  # 위도 (예: 서울)
    user_lon = 126.964794  # 경도 (예: 서울)


    # Elasticsearch 쿼리 실행
    es_results = query_pharmacy_elasticsearch(user_lat, user_lon)

    # 검색 결과 확인
    if "hits" in es_results and es_results['hits']['total']['value'] > 0:
        print(f"총 {es_results['hits']['total']['value']}개의 약국을 찾았습니다.")

        # 약국 데이터를 DataFrame으로 변환
        pharmacy_data = [hit['_source'] for hit in es_results['hits']['hits']]
        df = pd.DataFrame(pharmacy_data)

        # 소요시간 정보 추가
        df_with_times = add_travel_times_to_pharmacies(df, user_lat, user_lon)

        # 결과 출력
        print(df_with_times)
    else:
        print("조건에 맞는 약국이 없습니다.")

총 10000개의 약국을 찾았습니다.
  dutytime7s dutytime1c      dutytel1 dutytime3c                dutyetc  \
0       1100       2130   02-717-6415       2130  일요일은 홀수주만 운영(2,4주 휴무)   
1       None       1930   02-713-6590       1930                   None   
2       1000       2200   02-713-8716       2200                   None   
3       None       2000   02-712-1276       1800                   None   
4       None       2100  02-3273-0750       2100                   None   
5       None       1930  02-6949-3332       1930                   None   
6       None       2200   02-718-3011       2200                   None   
7       None       1900   02-712-2209       1900                   None   
8       None       1400   02-713-3757       1400                   None   
9       None       1800   02-715-6009       1800                   None   

  dutytime5c dutytime7c @version   wgs84lat dutytime2s  ...  \
0       2130       1900        1  37.545068       0830  ...   
1       1930       None    

In [24]:
df

,dutytime7s,dutytime1c,dutytel1,dutytime3c,dutyetc,dutytime5c,dutytime7c,@version,wgs84lat,dutytime2s,...,dutyaddr,@timestamp,wgs84lon,location,dutytime1s,dutytime3s,dutytime5s,travel_time_sec,travel_time_h,travel_time_min
0,1100,2130,02-717-6415,2130,"일요일은 홀수주만 운영(2,4주 휴무)",2130,1900,1,37.545068,0830,...,"서울특별시 용산구 청파로47길 66, (청파동2가)",2024-11-22T15:14:00.719544900Z,126.966770,"[126.9667705, 37.5450675]",0830,0830,0830,114.910,0.0,1.0
1,None,1930,02-713-6590,1930,None,1930,None,1,37.544699,0900,...,서울특별시 용산구 청파로47길 53,2024-11-22T15:14:00.975612Z,126.967399,"[126.9673986, 37.5446987]",0900,0900,0900,47.710,0.0,0.0
2,1000,2200,02-713-8716,2200,None,2200,2130,1,37.544818,0830,...,"서울특별시 용산구 청파로47길 40, (청파동2가)",2024-11-22T15:14:01.227788300Z,126.968269,"[126.9682689, 37.5448184]",0830,0830,0830,135.018,0.0,2.0
3,None,2000,02-712-1276,1800,None,2000,None,1,37.548356,0900,...,"서울특별시 용산구 효창원로93길 9, 한빛빌딩 2층 201호 (효창동)",2024-11-22T15:14:01.404092600Z,126.962832,"[126.9628323, 37.5483556]",0900,0900,0900,98.191,0.0,1.0
4,None,2100,02-3273-0750,2100,None,2100,None,1,37.548541,0900,...,"서울특별시 용산구 효창원로93길 2, (청파동2가)",2024-11-22T15:14:01.418148Z,126.963307,"[126.9633075, 37.5485414]",0900,0900,0900,89.915,0.0,1.0
5,None,1930,02-6949-3332,1930,None,1930,None,1,37.544706,0840,...,"서울특별시 용산구 청파로47길 7, 102호 (청파동3가)",2024-11-22T15:14:01.083758900Z,126.970224,"[126.9702237, 37.5447058]",0840,0840,0840,111.004,0.0,1.0
6,None,2200,02-718-3011,2200,None,2200,None,1,37.549368,0830,...,"서울특별시 마포구 효창원로97길 10, (공덕동)",2024-11-22T15:14:17.844980200Z,126.962999,"[126.9629995, 37.5493683]",0830,0830,0830,129.098,0.0,2.0
7,None,1900,02-712-2209,1900,None,1900,None,1,37.550009,0900,...,"서울특별시 마포구 효창원로 255-1, 1층 (공덕동)",2024-11-22T15:14:00.818061300Z,126.963465,"[126.963465, 37.5500091]",0900,0900,0900,120.373,0.0,2.0
8,None,1400,02-713-3757,1400,None,1400,None,1,37.540480,0800,...,"서울특별시 용산구 효창원로 142, (효창동)",2024-11-22T15:14:00.714321400Z,126.962001,"[126.9620008, 37.5404798]",0800,0800,0800,172.052,0.0,2.0
9,None,1800,02-715-6009,1800,None,1800,None,1,37.548576,0900,...,"서울특별시 용산구 청파로 324, 1층 (청파동1가)",2024-11-22T15:14:00.934467400Z,126.970247,"[126.9702469, 37.5485757]",0900,0900,0900,212.218,0.0,3.0
